(Visit the
[documentation](https://datafold-dev.gitlab.io/datafold/tutorial_index.html) page
to view the executed notebook.)

# Data structures: PCManifold and TSCDataFrame

This tutorial introduces the two data structures implemented in *datafold*.  

* `PCManifold` - point cloud manifolds  
* `TSCDataFrame` - time series collection as a special type of point cloud manifold  

Both classes are located in the package `pcfold`. They can be used in models and algorithms or as data input. Because both classes derive from classes that are widely used for data processing in Python (`numpy.ndarray` and `pandas.DataFrame`) the general handling is familiar for experienced Python users and we refer to the documentation of the original classes and packaged for a broader introduction. In the tutorial, we will only highlight in what context the two data structures are useful.  

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import numpy as np
import pandas as pd
import scipy
from scipy.sparse.linalg import lsqr
from sklearn.datasets import make_swiss_roll

from datafold.pcfold import PCManifold, TSCDataFrame
from datafold.pcfold.kernels import GaussianKernel

## 1. Point cloud on manifold (`PCManifold`)
`PCManifold` is derived from `numpy.ndarray` and therefore inherits the rich functionality of the quasi-standard data structure for numerical data in Python. `PCManifold` restricts the general purpose of the base class array to a specific case:

* A technical requirement is that the point cloud must be numeric (i.e. `dtype` cannot be `object`, `str` etc.) and must be a two-dimensional, where the single samples are orientated in rows and features in columns. 
* A non-technical requirement is that the point cloud is assumed to be sampled on a manifold. This means the points are sampled on an (unknown) geometrical structure. Usually, the all data should come from the same underlying process of system to fulfill this requirement. 

To showcase some of the functionality of `PCManifold`, we first generate data on a "swiss-roll manifold" using a generator function from scikit-learn. Once we have the point cloud, we create a new instance of `PCManifold` where we attach new attributes to the array: 

1. A kernel (here `GaussianKernel`) that describes the locality between point samples. 
2. An (optional) `cut_off` distance value, which defines a threshold on the pairwise point distances. All kernel values are set to zero if the corresponding distance exceeds the cut-off. The parameter allows us to promote sparsity by restricting the "sphere of influence" with respect to a chosen metric. The sparse kernel matrix can be better scaled and more samples can be included. In the example, we set the Euclidean distance and a `GaussianKernel`.
3. A distance backend to select an algorithm with which to compute the distance matrix. The backend has to support the specified metric in the kernel.

In [ ]:
X, color = make_swiss_roll(n_samples=2000)

pcm = PCManifold(
    X,
    kernel=GaussianKernel(epsilon=4),
    dist_kwargs=dict(cut_off=6, backend="guess_optimal"),
)

# plot the swiss roll dataset
fig = plt.figure(figsize=[7, 7])
ax = fig.add_subplot(1, 1, 1, projection="3d")
ax.scatter(*X.T, c=color, cmap=plt.cm.Spectral)
ax.set_title("Swiss roll: sampled manifold point cloud")

print(f"isinstance(pcm, np.ndarray)={isinstance(pcm, np.ndarray)}")
pcm  # displays the data

### Showcase: Radial basis interpolation of swiss-roll with color as function target 

We can now use the `PCManifold` object to evaluate the attached kernel and compute the kernel matrix for the actual point cloud. Kernel matrices are used in many algorithms with "manifold assumption", because the kernel describes the local information of a point with respect to its neighborhood. We showcase this by creating an radial basis interpolation (RBF) and use the (extended) functionality of `PCManifold`. For simplicity we take the (pseudo-)color values of the swiss-roll data generator as the function target values that we want to interpolate. 

In the first step we compute the pairwise kernel matrix. In the next step, having the kernel matrix and the known target values we compute the RBF weights by using a sparse least squares solver from the scipy package.

In [ ]:
# use PCManifold to evaluate specified kernel on point cloud
kernel_matrix = pcm.compute_kernel_matrix()  # returns a scipy.sparse.csr_matrix

# compute RBF interpolation weights
weights = lsqr(kernel_matrix, color)[0]
color_rbf_centers = kernel_matrix @ weights

# plotting:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(1, 1, 1, projection="3d")
ax.scatter(*X.T, c=color_rbf_centers, cmap=plt.cm.Spectral)
ax.set_title("RBF interpolation at training points");

The computed weights allow us to interpolate out-of-sample points with the RBF model. To actually interpolate points we generate a new set of points on the swiss-roll manifold, interpolate the color values and (visually) compare it with the true color information.  

The out-of-sample point cloud are a reference point cloud for the existing `PCManifold`. This means we compute the kernel matrix now component wise. Because we view the points independently for interpolation, we do not need to make new point cloud a `PCManifold` for the out-of-sample data.

In [ ]:
# create many out-of-sample points
X_interp, true_color = make_swiss_roll(20000)

# interpolate points with RBF model
kernel_matrix_interp = pcm.compute_kernel_matrix(
    Y=X_interp
)  # component wise if Y is not None
color_rbf_interp = kernel_matrix_interp @ weights

# plotting:
fig = plt.figure(figsize=(16, 9))

ax = fig.add_subplot(1, 2, 1, projection="3d")
ax.scatter(*X_interp.T, c=true_color, cmap=plt.cm.Spectral)
ax.set_title("True color values from swiss role")

ax = fig.add_subplot(1, 2, 2, projection="3d")
ax.scatter(*X_interp.T, c=color_rbf_interp, cmap=plt.cm.Spectral)
ax.set_title("Interpolated color at interpolated points");

### Summary

In effectively 4 lines of code we created an RBF interpolation by using the `PCManifold` data structure. We can now easily exchange a kernel, compute a kernel matrix with varying degree of sparsity, and choose a distance algorithm, which is usually the computationally most expensive part. The data structure makes kernel based algorithms much easier to write and improves code readability.

The showcase of RBF interpolation can be improved by 

* properly optimizing the kernel parameters (see e.g. `PCManifold.optimize_parameters()` or via cross validation)
* choose another interpolation method (e.g. `GeometricHarmonicsInterpolator`), as target values in regions with low sampling density quickly decrease to zero for RBF interpolation.

Because `PCManifold` inherits from `numpy.ndarray`, we can use all of NumPy's functionality in-place. For example, we can compute eigenvectors on a `PCManifold` with 

```
np.linalg.eig(pcm)
```

## 2. Time series collection (`TSCDataFrame`)

The data structure `TSCDataFrame` adds time context to data coming from dynamical systems. Examples can be sensor data but also the solution of an ODE/PDE system. The data-driven models aiming to learn a dynamical system from data, also known as ["system identification"](https://en.wikipedia.org/wiki/System_identification)), have often the assumption that the system's phase space is a manifold. However, in contrast to an unordered point cloud, time series data have an inherent temporal order. Furthermore, the whole available data may consist of a collection of time series, for example, sampled with different initial conditions). These "time issues" require often a separate handling compared to i.i.d. sampled point clouds.

To address the special handling of time series collection data, *datafold* provides the data structure `TSCDataFrame`. It subclasses from `pandas.DataFrame` and therefore inherits rich functionality from another popular Python package. The data stored in a `DataFrame` interoperates well with `numpy.ndarray`.


To showcase `TSCDataFrame` we define a simple two-dimensional linear system to generate (single) time series data as a `pandas.DataFrame`. Note, that the columns contain the spatial information (features) `x1` and `x2` and the index contains the time information. 

In [ ]:
def get_data(t, x0) -> pd.DataFrame:
    r"""Evaluate time series of randomly created linear system.

    Solves:

    .. code-block::

        dx/dt = A x

    where `A` is a random matrix.

    Parameters
    ----------
    t
        time values to evaluate

    x0
        initial state (2-dimensional)

    Returns
    -------
    pandas.DataFrame
        time series with shape `(n_time_values, 2)`
    """

    A = np.random.default_rng(1).standard_normal(size=(2, 2))

    expA = scipy.linalg.expm(A)
    states = np.row_stack(
        [scipy.linalg.fractional_matrix_power(expA, ti) @ x0 for ti in t]
    )

    return pd.DataFrame(data=np.real(states), index=t, columns=["x1", "x2"])

### Create a TSCDataFrame

Now that we have a way to generate individual time series, let us collect two of them into a `TSCDataFrame`. 

In general, we can create a new instance of `TSCDataFrame` in the same way as we would instantiate the superclass

```
DataFrame(data, index, columns, **kwargs)` 
```

However, when initializing a `TSCDataFrame`, certain requirements on the frame format must be fulfilled already. Otherwise, there will be an `AttributeError`. The special requirements of `TSCDataFrame` are:

* Two levels of a row-index, where the first index level is for the time series ID and the second index for the time values.
* One level column-index for the features.
* The time series IDs must be positive integers, and the time values must be non-negative numerical values. 
* No duplicated indexes are allowed, both in index and column.

The data orientation is the same as in `PCManifold`, this means single samples samples are row-wise and the features are columns. 

For easier instantiation, there exist class methods `TSCDataFrame.from_X`.  

Here, we use `TSCDataFrame.from_single_timeseries`, where we only need to insert a single `pandas.DataFrame(data, index=time, columns=feature_names)`. After the initial construction we can iteratively add new time series with `tsc.insert_ts()`.   

In [ ]:
# create a single time series as pandas data frame with time as index
x0 = np.random.randn(
    2,
)
x1 = np.random.randn(
    2,
)
data1 = get_data(np.arange(0, 5), x0)
data2 = get_data(np.arange(0, 5), x1)

# convert it to a "time series collection" (TSC) data frame
tsc_regular = TSCDataFrame.from_single_timeseries(data1)
tsc_regular = tsc_regular.insert_ts(
    data2
)  # here could be loop to insert more time series


print("delta_time:", tsc_regular.delta_time)
print("n_timesteps:", tsc_regular.n_timesteps)
print("is_const_delta_time:", tsc_regular.is_const_delta_time())
print("is_equal_length:", tsc_regular.is_equal_length())
print("is_same_time_values:", tsc_regular.is_same_time_values())

tsc_regular

We now create a second `TSCDataFrame`, in which the time series are not sharing the same time values. For instantiation, we use a different method `TSCDataFrame.from_frame_list`, which allows a list of single time series (as `pandas.DataFrame`) to be inserted. 

We see that `delta_time` and `n_timesteps` cannot give a "global" value of the entire time series collection anymore. Instead, the attributes list the value for each time series and is of type `pandas.Series`.

In [ ]:
df1 = get_data(
    np.arange(0, 5),
    np.random.randn(
        2,
    ),
)
df2 = get_data(
    np.arange(5, 10, 2),
    np.random.randn(
        2,
    ),
)

tsc_irregular = TSCDataFrame.from_frame_list([df1, df2])

print("delta_time:", tsc_irregular.delta_time)
print("")
print("n_timesteps:", tsc_irregular.n_timesteps)
print("")
print("is_const_delta_time:", tsc_irregular.is_const_delta_time())
print("is_equal_length:", tsc_irregular.is_equal_length())
print("is_same_time_values:", tsc_irregular.is_same_time_values())

# print the time series. It now has two series in it, with IDs 0 and 1.
tsc_irregular

### Accessing data

Because `TSCDataFrame` is a `pandas.DataFrame` most of the data access and functions work in the same way. However, there are a few things to consider:

* The `TSCDataFrame` type is kept as long as the accessed data slice is still valid (i.e. fulfils the special format requirements). This is also true if the sliced data would actually be a `Series` (but also note the last point in this list).
* If a slice leads to an invalid `TSCDataFrame`  then the general fallback type is `pandas.DataFrame` or `pandas.Series` (e.g. accessing a single row is an invalid time series because more than one sample is required).
* Currently, there are inconsistencies with pandas.DataFrame, because there is no "`TSCSeries`". This is most noticeable for `.iloc` slicing which returns `pandas.Series` even if the slice is a valid `TSCDataFrame` (with one column). A simple type conversion `TSCDataFrame(slice_result)` is the current workaround.

In the following, we look at some examples to slice data from the constructed `tsc_regular` and `tsc_irregular`.

#### Access an individual feature from the collection

Note that the type is now a `TSCDataFrame` and not a `pandas.Series`.

In [ ]:
slice_result = tsc_regular["x1"]

print(type(slice_result))
slice_result

It is also always possible to turn the object to a `pandas.DataFrame` beforehand. The returned value is now a `Series` and not a `DataFrame`.

In [ ]:
slice_result = pd.DataFrame(tsc_regular)["x1"]

print(type(slice_result))
slice_result

The inconsistency with `.iloc` slicing manifests as follows:

In [ ]:
slice_result = tsc_regular.iloc[:, 0]  # access the 0-th column

print(type(slice_result))
slice_result

Instead of having a `TSCDataFrame` as expected, we got a `Series`. In order to obtain a `TSCDataFrame` type we can simply initiate a new object with

In [ ]:
slice_result = TSCDataFrame(tsc_regular.iloc[:, 0])

print(type(slice_result))
slice_result

#### Access a single time series

A `TSCDataFrame` has a two-level index, the first index the ID and the second the time. When we now access a single time series ID, the now ID index is dropped; this is the expected behavior from the `pandas.DataFrame`. Because it is not a two level index anymore, this means that the returned slice is not a legal `TSCDataFrame` anymore. The fallback type is then a `pandas.DataFrame`.

In [ ]:
slice_result = tsc_regular.loc[0]

print(type(slice_result))
slice_result

#### Select specific time values

The minimum length of a time series is two. However, `TSCDataFrame` also supports single sampled time series and describes them as "degenerated time series". The advantage is a better interoperatibility with the superclass.

In the next step, we select certain time values and get the samples from each time series with a match. Note that the inherited rules of accessing data from a `pandas.DataFrame` hold. This means, in the example, not all requested time values have to exist in a time series (the time value 99 does not have a match with any time series). Only if *no* time value matches, a `KeyError` exception is raised.

In [ ]:
slice_result = tsc_irregular.select_time_values([3, 4, 5, 7, 99])
print(type(slice_result))
slice_result

Now, we only select a single time value, which has only one match. This is of course not a legal time series anymore, but the `TSCDataFrame` is still maintained. We can access all "degenerated" time series IDs with a `TSCDataFrame` method. 

In [ ]:
slice_result = tsc_irregular.select_time_values(1)
print(type(slice_result), "\n")
print("Degenerated IDs:", slice_result.degenerate_ids())
slice_result

#### Extracting initial states

Initial states are required for a dynamical model to make predictions and evolve the system forward in time. An initial condition can be either of a single state, but can also be a time series itself. The latter case occurs if the initial condition also consists of the current and the past samples. Extracting initial states can be achieved with the usual slicing of a `DataFrame`. Here we take the first sample by using the `groupby` function and take the first sample of each series:

In [ ]:
slice_result = tsc_regular.groupby("ID").head(1)
print(type(slice_result))
slice_result

The `TSCDataFrame` data structure also provides convenience methods:

In [ ]:
slice_result = tsc_regular.initial_states()
print(type(slice_result))
slice_result

The method also allows us to conveniently extract the first two samples of each time series. Note, however, that the time values mismatch:

In [ ]:
slice_result = tsc_irregular.initial_states(2)
print(type(slice_result))
slice_result

There is an extra class `InitialCodition` that provides methods and validation for initial conditions. 

For example, we want to address different situations:

* In the case where time series in a collection share the same time values, we can group them and evaluate these initial conditions together.

* If time series have different time values, we want to treat them separately and make separate predictions with the model.

This grouping functionality is very useful when we want to reconstruct time series data with a model. We use the iterator `InitialCondition.iter_reconstruct_ic` method:

(In the cell we also use `InitialCondition.validate(ic)` to check that the initial condition is valid.)

In [ ]:
from datafold.pcfold import InitialCondition

print("REGULAR CASE (groups time series together)")
print("------------------------------------------\n")

for ic, time_values in InitialCondition.iter_reconstruct_ic(tsc_regular):
    print(f"Initial condition \n")
    print(ic)
    assert InitialCondition.validate(ic)
    print(f"with corresponding time values {time_values}")


print(
    "\n\n==========================================================================\n\n"
)
print("IRREGULAR CASE (separates initial conditions):")
print("----------------------------------------------")

for ic, time_values in InitialCondition.iter_reconstruct_ic(tsc_irregular):
    print(f"Initial condition \n")
    print(ic)
    assert InitialCondition.validate(ic)
    print(f"with corresponding time values {time_values}\n\n")

### Plot time series data

`TSCDataFrame` provides basic plotting facility: 

In [ ]:
tsc_regular.plot(figsize=(7, 7))

We can also use the iterator `TSCDataFrmae.itertimeseries` which allows us access the time series separately and create plots for each time series. 

In [ ]:
f, ax = plt.subplots(1, len(tsc_regular.ids), figsize=(15, 7), sharey=True)

for _id, time_series in tsc_regular.itertimeseries():
    ts_axis = time_series.plot(ax=ax[_id])
    ts_axis.set_title(f"time series ID={_id}")
    if _id == 0:
        ts_axis.set_ylabel("quantity of interest")